In [198]:
import torch
from torch import nn

import pandas as pd

In [199]:
device = "cuda" if torch.cuda.is_available() else "cpu"

device

'cpu'

In [200]:
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fully_connected1 = nn.Linear(10, 5)
        self.fully_connected2 = nn.Linear(5, 2)

    def forward(self, x):
        x = torch.relu(self.fully_connected1(x))
        x = self.fully_connected2(x)
        return x

model = SimpleNeuralNet().to(device=device)
model

SimpleNeuralNet(
  (fully_connected1): Linear(in_features=10, out_features=5, bias=True)
  (fully_connected2): Linear(in_features=5, out_features=2, bias=True)
)

In [201]:
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]
    
data, labels = ["a", "b"], [1, 2]
dataset = CustomDataset(data, labels)
dataset

In [202]:
dataloader = DataLoader(dataset=dataset, batch_size=1, shuffle=True)

dataloader

In [203]:
for X, y in dataloader:
    print(X)
    print(y)
    break

('b',)
tensor([2])


# SimpleANN

In [204]:
import torch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import numpy as np

In [205]:
class RandomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [206]:
class SimpleANN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [207]:
def generate_data(samples=1000, input_size=2, num_classes=2):
    X = np.random.randn(samples, input_size)
    y = np.random.randint(0, num_classes, samples)
    return X, y

# Hyperparameters
samples=3000
input_size = 2
num_classes=2
hidden_size=16
output_size=2
learning_rate=0.01
epochs=100
batch_size=32

X, y = generate_data(samples=samples, input_size=input_size, num_classes=num_classes)

X.shape, y.shape

((3000, 2), (3000,))

In [208]:
X, y

(array([[ 0.54871764, -0.8431569 ],
        [ 0.07607584,  0.33348461],
        [ 0.79725423,  1.92790822],
        ...,
        [-1.65017794,  1.55940009],
        [ 0.74685988,  0.67317155],
        [-1.49636917,  0.13821293]]),
 array([0, 1, 1, ..., 0, 0, 0]))

In [209]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=52)

((X_train.shape, y_train.shape), (X_test.shape, y_test.shape))

(((2400, 2), (2400,)), ((600, 2), (600,)))

In [210]:
train_dataset = RandomDataset(X_train, y_train)
test_dataset = RandomDataset(X_test, y_test)

train_dataset, test_dataset, train_dataset.__len__(), test_dataset.__len__()

(<__main__.RandomDataset at 0x200f0c39ee0>,
 2400,
 600)

In [211]:
for X, y in train_dataset:
    print(X, y)
    print(X.dtype)
    break

tensor([-1.5226, -0.7125]) tensor(0)
torch.float32


In [212]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for X, y in train_dataset:
    print(X, X.dtype, y)
    break

train_dataset, test_dataset

tensor([-1.5226, -0.7125]) torch.float32 tensor(0)


(<__main__.RandomDataset at 0x200f0c39ee0>,
 <__main__.RandomDataset at 0x200f0c36030>)

In [213]:
device = "cuda" if torch.cuda.is_available() else "cpu"

device

'cpu'

In [214]:
model = SimpleANN(input_size=input_size, hidden_size=hidden_size, output_size=output_size).to(device=device)

model

SimpleANN(
  (fc1): Linear(in_features=2, out_features=16, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=16, out_features=2, bias=True)
)

In [215]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

loss, optimizer

(CrossEntropyLoss(),
 Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.01
     maximize: False
     weight_decay: 0
 ))

In [216]:
train_loader.dataset[0], len(train_loader), len(train_loader.dataset)

((tensor([-1.5226, -0.7125]), tensor(0)), 75, 2400)

In [217]:
def train(dataloader, model, loss_function, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_function(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch%200==0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"Loss: {loss}, current: {current}/{size}")


def inference(dataloader, model, loss_function):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            _, preds = torch.max(pred, 1)
            loss = loss_function(pred, y)
            test_loss += loss.item()
            correct += (preds == y).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f"Test accuracy: {correct*100}, Loss: {test_loss}")

In [218]:
for i in range(epochs):
    if i%5==0:
        print(f"Epoch: {i}")
        train(train_loader, model, loss_function=loss, optimizer=optimizer)
        inference(test_loader, model=model, loss_function=loss)

print("Done")

Epoch: 0
Loss: 0.7087156176567078, current: 32/2400
Test accuracy: 45.5, Loss: 0.6977007796889857
Epoch: 5
Loss: 0.7003018260002136, current: 32/2400
Test accuracy: 55.833333333333336, Loss: 0.690858498999947
Epoch: 10
Loss: 0.6964715123176575, current: 32/2400
Test accuracy: 47.333333333333336, Loss: 0.697541572545704
Epoch: 15
Loss: 0.7018226385116577, current: 32/2400
Test accuracy: 54.333333333333336, Loss: 0.6921732237464503
Epoch: 20
Loss: 0.6855232119560242, current: 32/2400
Test accuracy: 52.83333333333333, Loss: 0.6909546977595279
Epoch: 25
Loss: 0.712070643901825, current: 32/2400
Test accuracy: 53.333333333333336, Loss: 0.6917068707315546
Epoch: 30
Loss: 0.6675142049789429, current: 32/2400
Test accuracy: 52.33333333333333, Loss: 0.6915348485896462
Epoch: 35
Loss: 0.6965911984443665, current: 32/2400
Test accuracy: 48.66666666666667, Loss: 0.6921215590677763
Epoch: 40
Loss: 0.6912569999694824, current: 32/2400
Test accuracy: 52.83333333333333, Loss: 0.6910606591325057
Epoch: